In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
%cd sample_data
# !unzip newRespon_PCD.zip
# import shutil
# import os
# print(os.getcwd())
# shutil.rmtree('/content/sample_data/dataset_closed')

In [3]:
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import math
import os
import csv

In [4]:
labels = ['celana_panjang', 'celana_pendek', 'Kaos']
imgPerLabels = 160
orientation = [0, 45, 90, 135]
datasets = {}
input_folder = ['dataset1/', 'dataset_closed/']

for i in range(len(labels)):
        # if os.path.isdir(labels[j]+"\\"+str(imgindex)):
    datasets[labels[i]] = np.empty((imgPerLabels, len(input_folder), 256, 256))
    for j in range(len(input_folder)):
        count=0
        for filename in os.listdir(input_folder[j]):
            if(labels[i] in filename):
                img_path = os.path.join(input_folder[j], filename)
                img = cv.cvtColor(cv.imread(img_path), cv.COLOR_BGR2GRAY)
                datasets[labels[i]][count][j] = img
                count+=1

In [5]:
def getGlcm(datasets:np.ndarray, ori:np.ndarray[int, int], dist=1):
    result = {}

    for i in datasets:
        label = []
        for j in range(len(datasets[i])):
            imgs = []
            for k in range(len(datasets[i][j])):
                img:np.ndarray = datasets[i][j][k]
                imgGlcm = {}
                for l in range(len(ori)):
                    (iH, iW) = img.shape[:2]
                    matriks = np.zeros((256, 256), float)
                    startX = startY = 0
                    endX = iW
                    endY = iH

                    if ori[l]==0: endX=iW-dist
                    elif ori[l]==90: startY=dist
                    elif ori[l]==45:
                        startY=dist
                        endX=iW-dist
                    elif ori[l]==135:
                        startY=startX=dist

                    for y in range(startY, endY):
                        for x in range(startX, endX):
                            if ori[l]==0: matriks[int(img[y,x]), int(img[y,x+dist])] =+ 1
                            elif ori[l]==90: matriks[int(img[y,x]), int(img[y-dist,x])] =+ 1
                            elif ori[l]==45: matriks[int(img[y,x]), int(img[y-dist,x+dist])] =+ 1
                            elif ori[l]==135: matriks[int(img[y,x]), int(img[y-dist,x-dist])] =+ 1

                    matriks +=matriks.transpose()
                    summary = np.sum(matriks)
                    matriks /= summary
                    # for y in range(matriks.shape[0]):
                    #     for x in range(matriks.shape[1]):
                    #         if matriks[y,x] != 0: matriks[y,x] /= summary

                    imgGlcm[orientation[l]] = matriks
                imgs.append(imgGlcm)
            label.append(imgs)
        result[i] = label
    return result

In [6]:
glcm = getGlcm(datasets, orientation)

In [7]:
def getFeature(glcm:dict[int, dict[int, np.ndarray]], features:list[str]=['contrast', 'dissimiliarity', 'homogeneity', 'entropy', 'asm', 'corellation', 'energy']):

    result = []
    for i in glcm.keys():
        for j in range(len(glcm[i])):
            imgFeatures = {}
            for k in range(len(glcm[i][j])):
              for l in glcm[i][j][k].keys():
                matriks = glcm[i][j][k][l]

                fitur = {f"contrast{l}_{k}":0.0,
                            f"dissimiliarity{l}_{k}":0.0,
                            f"homogeneity{l}_{k}":0.0,
                            f"entropy{l}_{k}":0.0,
                            f"asm{l}_{k}":0.0,
                            f"corellation{l}_{k}":0.0,
                            f"energy{l}_{k}":0.0}

                meanY = np.mean(matriks)
                meanX = np.mean(matriks.transpose())

                for y in range(256):
                    for x in range(256):
                        if matriks[y,x]==0.0: continue
                        if "contrast" in features: fitur[f"contrast{l}_{k}"] += matriks[y,x]*math.pow(y-x, 2)
                        if "dissimiliarity" in features: fitur[f"dissimiliarity{l}_{k}"] += matriks[y,x]*abs(y-x)
                        if "homogeneity" in features: fitur[f"homogeneity{l}_{k}"] += matriks[y,x]/(1+math.pow(y-x, 2))
                        if "entropy" in features: fitur[f"entropy{l}_{k}"] -= matriks[y,x]*math.log10(matriks[y,x])
                        if "asm" in features: fitur[f"asm{l}_{k}"] += math.pow(matriks[y,x], 2)

                if "corellation" in features:
                    stdX = np.std(matriks)
                    stdY = np.std(matriks.transpose())
                    if stdX != 0 and stdY != 0: fitur[f"corellation{l}_{k}"] = (- (meanX * meanY)) / (stdX * stdY)
                if 'energy' in features: fitur[f"energy{l}_{k}"] = math.sqrt(fitur[f"asm{l}_{k}"])

                for m in fitur:
                    imgFeatures[m] = fitur[m]

            imgFeatures['label'] = i
            result.append(imgFeatures)
    return result

In [8]:
features = getFeature(glcm)

In [9]:
header = []

for head in features[0].keys():
    header.append(head)

with open('Features.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(features)

# Klasifikasi

In [10]:
import pandas as pd
import seaborn as sb

In [11]:
df = pd.read_csv('Features.csv')

In [12]:
df = df.replace ({
 'celana_panjang':0,
 'celana_pendek':1,
 'Kaos':2,
})
df.head()

,contrast0_0,dissimiliarity0_0,homogeneity0_0,entropy0_0,asm0_0,corellation0_0,energy0_0,contrast45_0,dissimiliarity45_0,homogeneity45_0,...,corellation90_2,energy90_2,contrast135_2,dissimiliarity135_2,homogeneity135_2,entropy135_2,asm135_2,corellation135_2,energy135_2,label
0,2050.086834,27.311805,0.073337,4.097881,0.000084,-0.220543,0.009189,2564.147867,30.643289,0.063978,...,-0.268953,0.008485,4060.568090,43.574660,0.043588,4.282325,0.000055,-0.380245,0.007442,0
1,523.326874,16.873500,0.063467,4.047713,0.000094,-0.194445,0.009682,620.868114,18.258295,0.059558,...,-0.225780,0.009102,3109.519126,39.393962,0.039366,4.275108,0.000056,-0.372441,0.007499,0
2,1055.799738,17.909614,0.087139,3.972092,0.000112,-0.158498,0.010561,1230.493287,20.532989,0.073651,...,-0.211449,0.009350,3506.560499,40.324959,0.042584,4.200372,0.000067,-0.296074,0.008173,0
3,1018.678072,22.600459,0.050991,4.237339,0.000061,-0.336197,0.007788,1236.183838,24.795609,0.047998,...,-0.343403,0.007726,2688.693747,36.812684,0.038563,4.379327,0.000044,-0.528743,0.006642,0
4,890.269529,19.537066,0.072879,4.057692,0.000092,-0.198916,0.009590,922.992808,21.174291,0.053367,...,-0.257199,0.008636,2366.585474,34.853136,0.043255,4.249887,0.000060,-0.344789,0.007715,0


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


x = df.iloc[:,:-1]
y = df.iloc[:,-1:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

scaler = MinMaxScaler()
x_train_scalled = scaler.fit_transform(x_train)
x_test_scalled = scaler.transform(x_test)

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
names = ["K-Nearest Neighbors",
         "Linear SVM"]

classifiers = [
    KNeighborsClassifier(5),
    SVC(gamma=2, C=1),
    ]

In [15]:
for name, model in zip(names, classifiers):
  model.fit(x_train_scalled, y_train.values.ravel())
  y_pred_model = model.predict(x_test_scalled)
  print(f'## MODEL NAME : {name}')
  print(confusion_matrix(y_test, y_pred_model))
  print(classification_report(y_test, y_pred_model))
  print('\n')

## MODEL NAME : K-Nearest Neighbors
[[27  2  0]
 [ 9 17  7]
 [ 7  9 18]]
              precision    recall  f1-score   support

           0       0.63      0.93      0.75        29
           1       0.61      0.52      0.56        33
           2       0.72      0.53      0.61        34

    accuracy                           0.65        96
   macro avg       0.65      0.66      0.64        96
weighted avg       0.65      0.65      0.63        96



## MODEL NAME : Linear SVM
[[25  2  2]
 [ 5 22  6]
 [ 3  9 22]]
              precision    recall  f1-score   support

           0       0.76      0.86      0.81        29
           1       0.67      0.67      0.67        33
           2       0.73      0.65      0.69        34

    accuracy                           0.72        96
   macro avg       0.72      0.73      0.72        96
weighted avg       0.72      0.72      0.72        96





In [16]:
correlation = {}
label = df['label']
for col in df.columns:
  correlation[col] = label.corr(df[col])
sorted_correlation = sorted(correlation.items(), key=lambda x: x[1])
print(sorted_correlation)
len(sorted_correlation)

[('dissimiliarity0_2', -0.41179724777115284), ('contrast0_2', -0.2860992708922338), ('entropy0_2', -0.2768948462683974), ('homogeneity90_1', -0.24384210463791706), ('homogeneity90_0', -0.1738234194527513), ('entropy135_2', -0.1665886187414108), ('entropy45_2', -0.16396911085864113), ('entropy0_0', -0.11503146751116418), ('entropy45_0', -0.08491011940634902), ('entropy135_0', -0.08431018147953417), ('homogeneity90_2', -0.07648909350948371), ('corellation90_1', -0.06196178253811031), ('entropy0_1', -0.060222701098059966), ('corellation90_0', -0.05164472653686163), ('dissimiliarity45_2', -0.051271142887835325), ('entropy135_1', -0.04834196956222339), ('entropy45_1', -0.04729171395418203), ('dissimiliarity135_2', -0.04058890847590595), ('homogeneity45_0', -0.038004172424222535), ('homogeneity135_0', -0.03754555228182002), ('corellation45_1', -0.034785127924821475), ('corellation135_1', -0.03392021229342949), ('dissimiliarity0_1', -0.03229149980045857), ('corellation0_1', -0.028803567128786

85

In [17]:
positive_corr = {}
negative_corr = {}
for (k,v) in sorted_correlation:
  if(v>0):
    positive_corr[k] = v
  else:
    negative_corr[k] = v
len(positive_corr)

56